<a href="https://colab.research.google.com/github/junweilam/ICT3104_T20/blob/main/ICT3104_Team20.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Testing

**Installing of necessary dependencies**

In [1]:
!pip install ipywidgets==7.6.3
!pip install pyyaml
!pip install opencv-python-headless

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.8/121.8 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 34.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 71.9 MB/s eta 0:00:00
  Attempting uninstall: widgetsnbextension
    Found existing installation: widgetsnbextension 3.6.6
    Uninstalling widgetsnbextension-3.6.6:
      Successfully uninstalled widgetsnbextension-3.6.6
  Attempting uninstall: ipywidgets
    Found existing installation: ipywidgets 7.7.1
    Uninstalling ipywidgets-7.7.1:
      Successfully uninstalled ipywidgets-7.7.1


In [2]:
!nvidia-smi --query-gpu=name,memory.total,memory.free --format=csv,noheader

Tesla T4, 15360 MiB, 15101 MiB


**Installation for Debugger**

In [3]:
#Debugger
!pip install -Uqq ipdb
import ipdb

**Debugger Switch (On/Off)**

In [4]:
# Turn On Debugger
%pdb on

# Turn Off Debugger
# %pdb off

Automatic pdb calling has been turned ON


# **Uploading Of Input Video File in .mp4 format** (T20-1)
1.   Run the cell and click the "Upload" button to upload a video in .mp4 format
2.   Once file is selected, click "Save MP4" button to save the .mp4 file
3.   File is saved when progress bar is 100% green and "Save MP4" button is changed to "MP4 Saved!"



In [5]:
# Testing.ipynb

import ipywidgets as widgets
from IPython.display import display
import os
import shutil
import asyncio
import platform
import time
from base64 import b64encode


# Create a file uploader widget
file_upload = widgets.FileUpload(
    accept='.mp4',
    multiple=False
)

# Create a button widget
save_button = widgets.Button(
    description="Save MP4",
    disabled=False,
    button_style='success'
)

# Create a progress widget
progress = widgets.FloatProgress(
    value=0,
    min=0,
    max=100,
    bar_style="success"
)

def update_progress_bar():
  progress.value += 25
  time.sleep(1)

def save_mp4(b):
    if file_upload.value:
        # Get the uploaded file
        uploaded_file = list(file_upload.value.values())[0]

        update_progress_bar()

        # Specify the destination folder to save the file
        destination_folder = './input/'

        update_progress_bar()

        # Create the destination folder if it doesn't exist
        if not os.path.exists(destination_folder):
            os.makedirs(destination_folder)

        update_progress_bar()

        with open(os.path.join(destination_folder, uploaded_file['metadata']['name']), 'wb') as f:
            f.write(uploaded_file['content'])

        update_progress_bar()

        # Disable the file uploaded and update the button text
        file_upload.disabled = True
        save_button.description = "MP4 Saved!"
    else:
        print("Please upload an MP4 file first")

display(file_upload, save_button, progress)

save_button.on_click(save_mp4)

FileUpload(value={}, accept='.mp4', description='Upload')

Button(button_style='success', description='Save MP4', style=ButtonStyle())

FloatProgress(value=0.0, bar_style='success')